<a href="https://colab.research.google.com/github/JoaoMMS98/ML_2.0/blob/main/Boruta_XGB_%2B_Permute_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boruta
!pip install Boruta-py

ERROR: Could not find a version that satisfies the requirement Boruta-py (from versions: none)
ERROR: No matching distribution found for Boruta-py


In [ ]:
from boruta import BorutaPy

# OR alternative import:
from boruta.boruta_py import BorutaPy

In [102]:
import pandas as pd

train_full = pd.read_csv('traindf (2).csv')
val_full = pd.read_csv('valdf (2).csv')
test_scaled = pd.read_csv('testdf (2).csv')

In [103]:
# # Divide datasets by 30 for speedtest
# train_full = train_full.iloc[::50, :]
# val_full = val_full.iloc[::50, :]
# test_scaled = test_scaled.iloc[::50, :]

In [104]:
merged_df = pd.concat([train_full, val_full], ignore_index=True)

In [105]:
merged_df

,Claim Injury Type,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,District Name,...,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M,wcio_pca1,wcio_pca2,region_cluster,high_risk_nature,high_risk_body,high_risk_cause,market_indicator
0,4. TEMPORARY,-0.067360,1.213328,-0.495974,-0.064860,-1.263841,-0.575548,0.983139,1.274579,1.046863,...,0,0,0,0.610477,0.806579,0.162263,1,0,0,1
1,3. MED ONLY,-0.908646,1.344678,0.955044,-0.783603,0.558209,-0.494377,-0.780636,0.384500,1.046863,...,0,0,0,-0.175047,0.713979,0.277415,0,0,1,1
2,2. NON-COMP,1.012027,-0.755254,-0.133220,0.864313,0.558209,-0.209935,0.983139,1.040655,1.046863,...,0,0,1,-0.025755,0.530618,1.125027,0,0,0,0
3,2. NON-COMP,0.056452,-0.755254,1.317798,0.041721,0.558209,-0.625062,-1.524399,1.040655,1.046863,...,0,0,1,-0.467459,-0.375868,-0.223555,0,0,0,0
4,3. MED ONLY,-0.788009,0.889468,0.156984,-0.698884,-1.896900,-0.453747,-1.524399,0.574898,1.046863,...,0,0,1,0.104639,0.995235,0.774823,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574020,4. TEMPORARY,0.173915,1.273293,0.229535,0.142837,0.558209,-0.518970,-0.780636,-1.416376,-1.261856,...,0,0,1,-1.601871,0.073562,-0.913279,0,0,0,0
574021,2. NON-COMP,0.085024,-0.755254,0.374636,0.060851,-1.026043,2.021605,-0.895805,1.274579,1.046863,...,0,0,0,-0.223769,1.753521,3.045807,0,0,0,0
574022,3. MED ONLY,-0.876899,-0.755254,-0.133220,-0.761740,0.558209,-0.622081,-0.780636,-0.959634,-0.667644,...,0,0,1,0.584317,1.572367,-0.040737,0,0,0,0
574023,4. TEMPORARY,1.666008,1.303004,-1.511687,1.421816,0.580704,-0.577546,0.983139,0.574898,1.046863,...,0,0,1,0.125125,0.021484,0.774823,0,0,0,0


In [106]:
# Create a copy of merged_df
cleaned_df = merged_df.copy()

# Drop the specified columns
cleaned_df = cleaned_df.drop(columns=[
    'Zip Code',
    'Alternative Dispute Resolution_U',
    'Claim Injury Type', # drop target
    'Agreement Reached',
    'County of Injury'
], errors='ignore')

# Keep only numeric columns
analysis_df = cleaned_df.select_dtypes(include=['number'])

# Similarly, drop columns from test_scaled
test_scaled = test_scaled.drop(columns=[
    'Zip Code',
    'Alternative Dispute Resolution_U',
    'Agreement Reached',
    'County of Injury'
], errors='ignore')

#Keep only numerical
test_scaled = test_scaled.select_dtypes(include=['number'])

In [107]:
analysis_df

,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,District Name,First Hearing Date,IME-4 Count,...,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M,wcio_pca1,wcio_pca2,region_cluster,high_risk_nature,high_risk_body,high_risk_cause,market_indicator
0,-0.067360,1.213328,-0.495974,-0.064860,-1.263841,-0.575548,0.983139,1.046863,-0.037533,0.920180,...,0,0,0,0.610477,0.806579,0.162263,1,0,0,1
1,-0.908646,1.344678,0.955044,-0.783603,0.558209,-0.494377,-0.780636,1.046863,-1.477658,-0.445038,...,0,0,0,-0.175047,0.713979,0.277415,0,0,1,1
2,1.012027,-0.755254,-0.133220,0.864313,0.558209,-0.209935,0.983139,1.046863,-0.037533,-0.445038,...,0,0,1,-0.025755,0.530618,1.125027,0,0,0,0
3,0.056452,-0.755254,1.317798,0.041721,0.558209,-0.625062,-1.524399,1.046863,-0.037533,-0.445038,...,0,0,1,-0.467459,-0.375868,-0.223555,0,0,0,0
4,-0.788009,0.889468,0.156984,-0.698884,-1.896900,-0.453747,-1.524399,1.046863,-1.413414,-0.445038,...,0,0,1,0.104639,0.995235,0.774823,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574020,0.173915,1.273293,0.229535,0.142837,0.558209,-0.518970,-0.780636,-1.261856,-0.037533,-0.445038,...,0,0,1,-1.601871,0.073562,-0.913279,0,0,0,0
574021,0.085024,-0.755254,0.374636,0.060851,-1.026043,2.021605,-0.895805,1.046863,-0.037533,-0.445038,...,0,0,0,-0.223769,1.753521,3.045807,0,0,0,0
574022,-0.876899,-0.755254,-0.133220,-0.761740,0.558209,-0.622081,-0.780636,-0.667644,-0.037533,-0.445038,...,0,0,1,0.584317,1.572367,-0.040737,0,0,0,0
574023,1.666008,1.303004,-1.511687,1.421816,0.580704,-0.577546,0.983139,1.046863,2.789179,3.650616,...,0,0,1,0.125125,0.021484,0.774823,0,0,0,0


In [108]:
test_scaled

,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,District Name,First Hearing Date,IME-4 Count,...,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M,wcio_pca1,wcio_pca2,region_cluster,high_risk_nature,high_risk_body,high_risk_cause,market_indicator
0,-1.686439,-0.755254,1.825655,-1.458620,0.558209,-0.411805,0.983139,1.046863,-0.037533,-0.445038,...,0,0,1,0.343719,-1.434468,1.148770,0,0,0,0
1,-1.686439,-0.755254,1.825655,-1.458620,0.558209,-0.362470,0.983139,1.046863,-0.037533,-0.445038,...,0,0,0,-0.240232,-1.024995,0.340333,0,0,0,0
2,-1.686439,-0.755254,-1.076382,-1.453154,0.558209,-0.623482,0.983139,1.046863,-0.037533,-0.445038,...,0,0,0,-1.204931,-0.950851,-0.671104,0,0,0,0
3,-1.686439,-0.755254,-143.493821,-1.458620,0.558209,-0.283982,0.983139,1.046863,-0.037533,-0.445038,...,0,0,0,0.521399,1.395544,-0.240175,0,0,0,0
4,-1.686439,-0.755254,1.390349,-1.453154,0.558209,-0.322674,0.983139,1.046863,-0.037533,-0.445038,...,0,0,1,-0.952494,-0.405208,1.347021,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387970,-3.337265,-0.755254,-1.294035,8.715824,0.558209,2.021605,-0.895805,-1.049064,-0.037533,-0.445038,...,0,0,1,-1.032355,-0.252428,-0.706718,1,0,1,1
387971,-3.337265,-0.755254,-0.931280,0.183830,-4.483761,-0.430883,-0.780636,-0.901458,-0.037533,-0.445038,...,0,0,0,-1.032355,-0.252428,0.091035,1,0,1,1
387972,-3.337265,-0.755254,0.084433,0.183830,0.558209,2.021605,-0.895805,1.046863,-0.037533,-0.445038,...,0,0,1,-1.032355,-0.252428,1.399254,1,0,1,1
387973,-3.337265,-0.755254,0.229535,0.183830,-4.461267,-0.453747,-1.524399,1.046863,-0.037533,-0.445038,...,0,0,1,-1.032355,-0.252428,0.162263,1,0,1,1


In [ ]:
y = merged_df['Claim Injury Type']


In [ ]:
analysis_df['Accident Month'] = analysis_df['Accident Month'].astype(float) # If 'Accident Month' is not already numeric

In [ ]:
import numpy as np

columns_to_impute = ['Accident Month', 'Accident Day', 'Days_to_Assembly', 'Under_20', 'Age_21_40', 'Age_41_65', 'Above_65', 'Accident Month Sin', 'Accident Month Cos', 'Accident Day Sin', 'Accident Day Cos']

for column in columns_to_impute:  # Correct indentation here
    median_value = analysis_df[analysis_df[column] != 0][column].median()
    analysis_df[column] = analysis_df[column].replace(0, median_value)
    test_scaled[column] = test_scaled[column].replace(0, median_value)
    test_scaled[column] = test_scaled[column].replace(0, median_value)
    X[column] = X[column].replace(0, median_value)

In [109]:
X = analysis_df
y = merged_df['Claim Injury Type']

Boruta is an all-relevant feature selection method that aims to identify all features that have a significant impact on the target variable, rather than just a minimal subset. This is particularly useful in complex datasets where multiple features may contribute to the outcome.

Because of previous runs with RFE and at every round the changing F1 score that per drop would go up and down I became critical of it's effectiveness because it may drop the least important feature but it was not giving me a 'clear' idea of the best combination of features so I started considering it more of a lesser method compared to something that could offer more insights in combinations of features rather than a simple dropping method. I found Boruta on Kaggle blogs and considered it a potentially more usefull tool for the insight i was looking for.

In [82]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from boruta import BorutaPy
import numpy as np
import pandas as pd

def run_boruta_feature_selection(X, y):
    """
    Perform Boruta feature selection.
    """
    # Label encode the target variable
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # Impute missing values
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)

    # Split data with stratification
    X_train, X_val, y_train, y_val = train_test_split(
        X_imputed, y_encoded,
        test_size=0.3,
        random_state=42,
        stratify=y_encoded
    )

    # Configure XGBoost classifier
    xgb_classifier = XGBClassifier(
          objective='multi:softprob',
          random_state=42,
          learning_rate=0.075,
          max_depth=18,      #
          min_child_weight=8,
          subsample=0.8,
          colsample_bytree=0.6,
          n_estimators=2000,
          reg_alpha=1.0,
          reg_lambda=1.5,
    )

    # Create Boruta feature selector
    boruta_selector = BorutaPy(
        estimator=xgb_classifier,
        n_estimators=100,
        max_iter=200,
        random_state=42,
        verbose=2,
        perc=75
    )

    # Fit Boruta selector
    boruta_selector.fit(X_train, y_train)

    # Prepare results
    selected_mask = boruta_selector.support_
    results = {
        'selected_features': X.columns[selected_mask].tolist(),
        'selected_features_count': np.sum(selected_mask),
        'tentative_features': X.columns[boruta_selector.support_weak_].tolist(),
        'tentative_features_count': np.sum(boruta_selector.support_weak_),
        'rejected_features': X.columns[~selected_mask].tolist(),
        'rejected_features_count': np.sum(~selected_mask),
        'ranking': boruta_selector.ranking_,
        'label_encoding': {
            original: encoded
            for original, encoded in zip(label_encoder.classes_, range(len(label_encoder.classes_)))
        }
    }

    return results

def main():
    # Use the existing X and y from the calling context
    boruta_results = run_boruta_feature_selection(X, y)

    # Print results
    print("Boruta Feature Selection Results:")
    print(f"Total selected features: {boruta_results['selected_features_count']}")

    print("\nSelected Features:")
    for feature in boruta_results['selected_features']:
        print(feature)

    print("\nLabel Encoding:")
    for original, encoded in boruta_results['label_encoding'].items():
        print(f"{original} -> {encoded}")

    return boruta_results

if __name__ == '__main__':
    main()

Iteration: 	1 / 200
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	2 / 200
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	3 / 200
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	4 / 200
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	5 / 200
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	6 / 200
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	7 / 200
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	8 / 200
Confirmed: 	30
Tentative: 	2
Rejected: 	9
Iteration: 	9 / 200
Confirmed: 	30
Tentative: 	2
Rejected: 	9
Iteration: 	10 / 200
Confirmed: 	30
Tentative: 	2
Rejected: 	9
Iteration: 	11 / 200
Confirmed: 	30
Tentative: 	2
Rejected: 	9
Iteration: 	12 / 200
Confirmed: 	30
Tentative: 	2
Rejected: 	9
Iteration: 	13 / 200
Confirmed: 	30
Tentative: 	2
Rejected: 	9
Iteration: 	14 / 200
Confirmed: 	30
Tentative: 	2
Rejected: 	9
Iteration: 	15 / 200
Confirmed: 	30
Tentative: 	2
Rejected: 	9
Iteration: 	16 / 200
Confirmed: 	30
Tentative: 	2
Rejected: 	9
I

In [110]:
print("Selected features:", selected_features)
print("Available columns in analysis_df:", analysis_df.columns)
print("Available columns in test_scaled:", test_scaled.columns)

Selected features: selected_features
Available columns in analysis_df: Index(['Assembly Date', 'Average Weekly Wage', 'Birth Year', 'C-2 Date',
       'C-3 Date', 'Carrier Name', 'Carrier Type', 'District Name',
       'First Hearing Date', 'IME-4 Count', 'Industry Code',
       'WCIO Cause of Injury Code', 'WCIO Nature of Injury Code',
       'WCIO Part Of Body Code', 'Number of Dependents', 'Accident Year',
       'Accident Month', 'Accident Day', 'Days_to_Assembly', 'Under_20',
       'Age_21_40', 'Age_41_65', 'Above_65', 'Accident Month Sin',
       'Accident Month Cos', 'Accident Day Sin', 'Accident Day Cos',
       'Received_C2', 'Received_C3', 'Hearing_held',
       'Attorney/Representative_Y', 'COVID-19 Indicator_Y',
       'Alternative Dispute Resolution_Y', 'Gender_M', 'wcio_pca1',
       'wcio_pca2', 'region_cluster', 'high_risk_nature', 'high_risk_body',
       'high_risk_cause', 'market_indicator'],
      dtype='object')
Available columns in test_scaled: Index(['Assembly D

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, classification_report


In [ ]:
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, classification_report
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import numpy as np
from optuna import TrialPruned
from sklearn.model_selection import train_test_split

In [83]:
selected_features = 'selected_features'

In [111]:
selected_hardcode = [
    'Assembly Date',
    'Average Weekly Wage',
    'Birth Year',
    'C-2 Date',
    'C-3 Date',
    'Carrier Name',
    'Carrier Type',
    'District Name',
    'First Hearing Date',
    'IME-4 Count',
    'Industry Code',
    'WCIO Cause of Injury Code',
    'WCIO Nature of Injury Code',
    'WCIO Part Of Body Code',
    'Accident Year',
    'Accident Month',
    'Received_C2',
    'Received_C3',
    'Hearing_held',
    'Attorney/Representative_Y',
    'COVID-19 Indicator_Y',
    'Alternative Dispute Resolution_Y',
    'Gender_M',
    'wcio_pca1',
    'wcio_pca2',
    'region_cluster',
    'high_risk_nature',
    'high_risk_body',
    'high_risk_cause',
    'market_indicator'
]

In [112]:
analysis_df = analysis_df[[col for col in selected_hardcode if col in analysis_df.columns]]
test_scaled = test_scaled[[col for col in selected_hardcode if col in test_scaled.columns]]

In [113]:
y = merged_df['Claim Injury Type']
claim_injury_type = merged_df['Claim Injury Type']

In [ ]:
def process_and_predict(analysis_df, claim_injury_type, test_scaled):
    """
    Implementation with feature importance and additional performance metrics.
    """
    # Label encoding
    le = LabelEncoder()
    y = le.fit_transform(claim_injury_type)

    # Stratified split
    X_train, X_val, y_train, y_val = train_test_split(
        analysis_df, y,
        test_size=0.3,
        random_state=42,
        stratify=y
    )

    model = XGBClassifier(
        objective='multi:softprob',
        random_state=42,
        learning_rate=0.07037445421868485,
        max_depth=11,
        min_child_weight=9.916499756210175,
        subsample=0.6870540499767197,
        colsample_bytree=0.7793210632845388,
        n_estimators=907,
        reg_alpha=0.09201671350123276,
        reg_lambda=1.554753386470569,
        gamma=1.6255768906192565,
    )

    # Model fitting
    model.fit(X_train, y_train)

    # Predictions
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_val)
    test_pred = model.predict(test_scaled)

    # Probability predictions for ROC AUC
    train_pred_proba = model.predict_proba(X_train)
    val_pred_proba = model.predict_proba(X_val)

    # Feature importance calculation
    feature_importance = pd.DataFrame({
        'feature': analysis_df.columns,
        'importance': model.feature_importances_
    })
    feature_importance = feature_importance.sort_values('importance', ascending=False)

    # Expanded metrics
    metrics = {
        'train_f1': f1_score(y_train, train_pred, average='macro'),
        'val_f1': f1_score(y_val, val_pred, average='macro'),
        'train_acc': accuracy_score(y_train, train_pred),
        'val_acc': accuracy_score(y_val, val_pred),
        'train_precision': precision_score(y_train, train_pred, average='macro'),
        'val_precision': precision_score(y_val, val_pred, average='macro'),
        'train_recall': recall_score(y_train, train_pred, average='macro'),
        'val_recall': recall_score(y_val, val_pred, average='macro'),
        'train_roc_auc': roc_auc_score(y_train, train_pred_proba, multi_class='ovr'),
        'val_roc_auc': roc_auc_score(y_val, val_pred_proba, multi_class='ovr')
    }

    # Classification reports
    print("\nTRAINING SET PERFORMANCE METRICS:")
    print(classification_report(y_train, train_pred))
    print("\nVALIDATION SET PERFORMANCE METRICS:")
    print(classification_report(y_val, val_pred))

    # Feature importance display
    print("\nFEATURE IMPORTANCE RANKING:")
    print(feature_importance.to_string(index=False))

    # Class distribution analysis
    print("\nCLASS DISTRIBUTION ANALYSIS:")
    print("Training Set Class Distribution:")
    print(pd.Series(y_train).value_counts(normalize=True))
    print("\nValidation Set Class Distribution:")
    print(pd.Series(y_val).value_counts(normalize=True))

    # Transform predictions
    test_pred_labels = le.inverse_transform(test_pred)

    return {
        'metrics': metrics,
        'test_predictions': test_pred_labels,
        'model': model,
        'label_encoder': le,
        'feature_importance': feature_importance
    }

# Execute analysis
try:
    results = process_and_predict(analysis_df, claim_injury_type, test_scaled)

    print("\nPERFORMANCE METRICS SUMMARY:")
    print(f"Train Accuracy: {results['metrics']['train_acc']:.4f}")
    print(f"Validation Accuracy: {results['metrics']['val_acc']:.4f}")
    print(f"Train F1-macro: {results['metrics']['train_f1']:.4f}")
    print(f"Validation F1-macro: {results['metrics']['val_f1']:.4f}")
    print(f"Train Precision: {results['metrics']['train_precision']:.4f}")
    print(f"Validation Precision: {results['metrics']['val_precision']:.4f}")
    print(f"Train Recall: {results['metrics']['train_recall']:.4f}")
    print(f"Validation Recall: {results['metrics']['val_recall']:.4f}")
    print(f"Train ROC AUC: {results['metrics']['train_roc_auc']:.4f}")
    print(f"Validation ROC AUC: {results['metrics']['val_roc_auc']:.4f}")

    # Save predictions
    predictions_df = pd.DataFrame({
        'Predicted_Claim_Injury_Type': results['test_predictions']
    })
    predictions_df.to_csv('test_predictions3.csv', index=False)

    # Save feature importance
    results['feature_importance'].to_csv('feature_importance3.csv', index=False)

except Exception as e:
    print(f"Error detected: {str(e)}")
    raise

In [92]:
print("Analysis DataFrame shape:", analysis_df.shape if 'analysis_df' in locals() else "Not found")
print("Claim injury type length:", len(claim_injury_type) if 'claim_injury_type' in locals() else "Not found")
print("Test scaled shape:", test_scaled.shape if 'test_scaled' in locals() else "Not found")

Analysis DataFrame shape: (574025, 0)
Claim injury type length: 574025
Test scaled shape: (387975, 0)


In [93]:
print("Analysis DataFrame type:", type(analysis_df) if 'analysis_df' in locals() else "Not found")
print("Claim injury type type:", type(claim_injury_type) if 'claim_injury_type' in locals() else "Not found")
print("Test scaled type:", type(test_scaled) if 'test_scaled' in locals() else "Not found")

Analysis DataFrame type: <class 'pandas.core.frame.DataFrame'>
Claim injury type type: <class 'pandas.core.series.Series'>
Test scaled type: <class 'pandas.core.frame.DataFrame'>


In [ ]:
!pip install kaggle

In [ ]:
import os
import json
import pandas as pd
import subprocess

# Create Kaggle credentials
credentials = {"username":"chielgroeneveld","key":"a823db81aa0c4aa800e0d8ad825fa855"}

# Ensure directory exists and write credentials
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump(credentials, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

# Read the files and create submission
sample_submission = pd.read_csv('sample_submission.csv')
test_predictions = pd.read_csv('test_predictions3.csv')

# Copy predictions into sample submission format
sample_submission['Claim Injury Type'] = test_predictions['Predicted_Claim_Injury_Type']

# Save as submission file
sample_submission.to_csv('submission.csv', index=False)

# Submit to Kaggle
try:
    result = subprocess.run(
        'kaggle competitions submit -c to-grant-or-not-to-grant -f submission.csv -m "group 19 submission"',
        shell=True,
        capture_output=True,
        text=True,
        check=True
    )
    print("Submission Output:", result.stdout)
    print("Submission Successful!")
except subprocess.CalledProcessError as e:
    print("Error:", e.stderr)
    print("Return Code:", e.returncode)